In [ ]:
# -----------------------------
# Import required libraries
# -----------------------------

import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from mlflow.tracking import MlflowClient


# -----------------------------
# Configure MLflow
# -----------------------------

# End any active runs from previous executions
mlflow.end_run()

# Set the MLflow Tracking Server URL (where the UI is running)
mlflow.set_tracking_uri("http://127.0.0.1:5000")

#tracking URL for mlflow deployed in ec2
#mlflow.set_tracking_uri("http://ec2-3-110-45-123.ap-south-1.compute.amazonaws.com:5000")

mlflow.set_tag("experiment_type", "dev")
mlflow.set_tag("model_type", "RandomForestClassifier")

# Create or select an experiment in MLflow
# All runs will be stored under this experiment name
mlflow.set_experiment("15thfeb_mlflow_exp")

# -----------------------------
# Load and prepare the dataset
# -----------------------------

# Load the Iris dataset
# X = input features, y = target labels
X, y = load_iris(return_X_y=True)

# Split the data into training and testing sets
# 80% for training and 20% for testing
# the value of random_state can't be negative
#In Scikit-learn, it controls the shuffling applied to the data before applying the split. We use it in train_test_split for splitting data into training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# Define hyperparameter values to try
# -----------------------------

# Different values for number of trees in the forest
n_estimators_list = [10, 20, 50, 100, 200]

# Different values for maximum depth of each tree
max_depth_list = [1, 2,3, 5, 10]

# -----------------------------
# Run multiple experiments using loops
# -----------------------------

# Loop over all combinations of n_estimators and max_depth
# Each combination will create one MLflow run
for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:

        # Start a new MLflow run
        with mlflow.start_run():

            # -----------------------------
            # Create and train the model
            # -----------------------------

            # Create a Random Forest model with current hyperparameters
            model = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                random_state=42
            )

            # Train the model using training data
            model.fit(X_train, y_train)

            # -----------------------------
            # Evaluate the model
            # -----------------------------

            # Make predictions on the test dataset
            y_pred = model.predict(X_test)

            # Calculate accuracy of the model
            accuracy = accuracy_score(y_test, y_pred)

            # -----------------------------
            # Log parameters, metrics, and model to MLflow
            # -----------------------------

            # Log the hyperparameters used in this run
            mlflow.log_param("n_estimators", n_estimators)
            mlflow.log_param("max_depth", max_depth)

            # Log the evaluation metric (accuracy)
            mlflow.log_metric("accuracy", accuracy)

            # Log (save) the trained model as an MLflow artifact
            mlflow.sklearn.log_model(model, "model")

            # -----------------------------
            # Print result for this run
            # -----------------------------

            print(
                f"Completed run: n_estimators={n_estimators}, "
                f"max_depth={max_depth}, accuracy={accuracy}"
            )




# -----------------------------
# Configure MLflow
# -----------------------------
EXPERIMENT_NAME = "15thfeb_mlflow_exp"
MODEL_NAME = "IrisBestModel"

client = MlflowClient()

# -----------------------------
# Find the best run
# -----------------------------
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

if exp is None:
    raise Exception(f"❌ Experiment '{EXPERIMENT_NAME}' not found.")

runs = client.search_runs(
    experiment_ids=[exp.experiment_id],
    order_by=[
        "metrics.accuracy DESC",
        "attributes.start_time DESC"
    ],
    max_results=1
)

if len(runs) == 0:
    raise Exception("❌ No runs found in this experiment. Check tracking URI or experiment name.")

best_run = runs[0]

print("🏆 Best Run Selected!")
print("Run ID     :", best_run.info.run_id)
print("Run Name   :", best_run.data.tags.get("mlflow.runName"))
print("Accuracy   :", best_run.data.metrics["accuracy"])
print("Parameters :", best_run.data.params)

# -----------------------------
# Load the best model --> Give me the model to use
# -----------------------------
best_model = mlflow.sklearn.load_model(f"runs:/{best_run.info.run_id}/model")
print("✅ Best model loaded:", best_model)

# -----------------------------
# Register the best model --> Make this model an official, versioned, deployable asset
# -----------------------------
print("📦 Registering the best model...")

result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=MODEL_NAME
)

print("✅ Model registered!")
print("Model name   :", result.name)
print("Model version:", result.version)